# Revenue Loss Due to Downgrades & Churn. 

Business Problem Customers are either downgrading plans or cancelling, leading to revenue leakage. 

Key Questions How much Monthly Recurring Revenue (MRR) is lost due to churn? 

Which plans experience frequent downgrades? 

Are pricing or features causing dissatisfaction?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
subs = pd.read_csv('../data/ravenstack_subscriptions.csv')
subs

,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,downgrade_flag,churn_flag,billing_frequency,auto_renew_flag
0,S-8cec59,A-3c1a3f,2023-12-23,2024-04-12,Enterprise,14,2786,33432,False,False,False,True,monthly,True
1,S-0f6f44,A-9b9fe9,2024-06-11,NaN,Pro,17,833,9996,False,False,False,False,monthly,True
2,S-51c0d1,A-659280,2024-11-25,NaN,Enterprise,62,0,0,True,True,False,False,annual,False
3,S-f81687,A-e7a1e2,2024-11-23,2024-12-13,Enterprise,5,995,11940,False,False,False,True,monthly,True
4,S-cff5a2,A-ba6516,2024-01-10,NaN,Enterprise,27,5373,64476,False,False,False,False,monthly,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,S-c158df,A-fa2041,2024-10-27,NaN,Enterprise,18,3582,42984,False,False,False,False,monthly,True
4996,S-b49c0a,A-db5e9e,2024-12-05,NaN,Pro,23,1127,13524,False,False,False,False,monthly,False
4997,S-42822d,A-443f6f,2024-12-15,NaN,Pro,25,1225,14700,False,False,False,False,annual,True
4998,S-432401,A-c7ffc2,2023-05-25,NaN,Pro,132,6468,77616,False,False,False,False,monthly,True


In [4]:
churn = pd.read_csv('../data/ravenstack_churn_events.csv')
churn

,churn_event_id,account_id,churn_date,reason_code,refund_amount_usd,preceding_upgrade_flag,preceding_downgrade_flag,is_reactivation,feedback_text
0,C-816288,A-c37cab,2024-10-27,pricing,4.03,False,False,False,switched to competitor
1,C-5a81e7,A-37f969,2024-06-25,support,96.45,True,False,False,NaN
2,C-a174be,A-b07346,2024-11-12,budget,0.00,False,False,False,missing features
3,C-accb39,A-1e50e0,2023-11-01,budget,54.94,False,False,False,switched to competitor
4,C-92f889,A-956988,2024-12-30,unknown,0.00,False,True,True,too expensive
...,...,...,...,...,...,...,...,...,...
595,C-b56a35,A-702032,2024-06-14,competitor,0.00,False,False,False,switched to competitor
596,C-7e7822,A-dbc825,2024-02-03,budget,0.00,False,False,False,NaN
597,C-484fba,A-0a282f,2024-12-31,support,62.66,False,True,False,too expensive
598,C-22f536,A-e5d6ab,2024-05-11,competitor,0.00,True,False,False,NaN


In [5]:
account = pd.read_csv('../data/ravenstack_accounts.csv')
account

,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
0,A-2e4581,Company_0,EdTech,US,2024-10-16,partner,Basic,9,False,False
1,A-43a9e3,Company_1,FinTech,IN,2023-08-17,other,Basic,18,False,True
2,A-0a282f,Company_2,DevTools,US,2024-08-27,organic,Basic,1,False,False
3,A-1f0ac7,Company_3,HealthTech,UK,2023-08-27,other,Basic,24,True,False
4,A-ce550d,Company_4,HealthTech,US,2024-10-27,event,Enterprise,35,False,True
...,...,...,...,...,...,...,...,...,...,...
495,A-8ae3fc,Company_495,DevTools,CA,2024-06-28,ads,Pro,9,False,False
496,A-55f257,Company_496,FinTech,US,2023-12-21,organic,Basic,9,False,False
497,A-d26ab4,Company_497,DevTools,UK,2024-11-07,organic,Basic,9,False,True
498,A-712533,Company_498,EdTech,US,2023-07-31,organic,Pro,18,False,False


In [6]:
df = pd.concat([subs, churn, account], axis= 1)
df

,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,...,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
0,S-8cec59,A-3c1a3f,2023-12-23,2024-04-12,Enterprise,14,2786,33432,False,False,...,A-2e4581,Company_0,EdTech,US,2024-10-16,partner,Basic,9.0,False,False
1,S-0f6f44,A-9b9fe9,2024-06-11,NaN,Pro,17,833,9996,False,False,...,A-43a9e3,Company_1,FinTech,IN,2023-08-17,other,Basic,18.0,False,True
2,S-51c0d1,A-659280,2024-11-25,NaN,Enterprise,62,0,0,True,True,...,A-0a282f,Company_2,DevTools,US,2024-08-27,organic,Basic,1.0,False,False
3,S-f81687,A-e7a1e2,2024-11-23,2024-12-13,Enterprise,5,995,11940,False,False,...,A-1f0ac7,Company_3,HealthTech,UK,2023-08-27,other,Basic,24.0,True,False
4,S-cff5a2,A-ba6516,2024-01-10,NaN,Enterprise,27,5373,64476,False,False,...,A-ce550d,Company_4,HealthTech,US,2024-10-27,event,Enterprise,35.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,S-c158df,A-fa2041,2024-10-27,NaN,Enterprise,18,3582,42984,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,S-b49c0a,A-db5e9e,2024-12-05,NaN,Pro,23,1127,13524,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,S-42822d,A-443f6f,2024-12-15,NaN,Pro,25,1225,14700,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,S-432401,A-c7ffc2,2023-05-25,NaN,Pro,132,6468,77616,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

In [7]:
df.info() # information about the data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   subscription_id           5000 non-null   object 
 1   account_id                5000 non-null   object 
 2   start_date                5000 non-null   object 
 3   end_date                  486 non-null    object 
 4   plan_tier                 5000 non-null   object 
 5   seats                     5000 non-null   int64  
 6   mrr_amount                5000 non-null   int64  
 7   arr_amount                5000 non-null   int64  
 8   is_trial                  5000 non-null   bool   
 9   upgrade_flag              5000 non-null   bool   
 10  downgrade_flag            5000 non-null   bool   
 11  churn_flag                5000 non-null   bool   
 12  billing_frequency         5000 non-null   object 
 13  auto_renew_flag           5000 non-null   bool   
 14  churn_ev

In [8]:
df.isnull() # check the missing value

,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,...,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,False,False,False,True,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
4996,False,False,False,True,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
4997,False,False,False,True,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
4998,False,False,False,True,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True


In [9]:
# fill the missing value inside the numeric column.

df.fillna(df.mean(numeric_only= True), inplace= True) # fill the numeric column value using fillna() and dropna().
df.dropna(inplace= True)

In [10]:
# fill missing value in categorical column using mode.

categorical_cols = df.select_dtypes(include= ['object']).columns

for col in categorical_cols:
    m = df[col].mode(dropna= True) 
    if not m.empty:
        df[col] = df[col].fillna(m.iloc[0])

C:\Users\Hp\AppData\Local\Temp\ipykernel_21416\4245578170.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(m.iloc[0])


In [11]:
df.drop_duplicates(inplace= True) # remove the duplicates value in the dataset

In [12]:
df

,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,...,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
0,S-8cec59,A-3c1a3f,2023-12-23,2024-04-12,Enterprise,14,2786,33432,False,False,...,A-2e4581,Company_0,EdTech,US,2024-10-16,partner,Basic,9.0,False,False
3,S-f81687,A-e7a1e2,2024-11-23,2024-12-13,Enterprise,5,995,11940,False,False,...,A-1f0ac7,Company_3,HealthTech,UK,2023-08-27,other,Basic,24.0,True,False
16,S-13939b,A-86902e,2023-07-21,2024-08-03,Pro,34,0,0,True,False,...,A-463db0,Company_16,HealthTech,US,2024-12-05,partner,Basic,15.0,False,False
17,S-9333c5,A-ef84cf,2024-06-06,2024-08-13,Basic,8,152,1824,False,False,...,A-ce66f8,Company_17,EdTech,CA,2023-08-23,event,Pro,29.0,False,False
21,S-3bd427,A-2e3bad,2024-12-04,2024-12-27,Basic,61,1159,13908,False,False,...,A-c42f1f,Company_21,HealthTech,US,2024-07-28,other,Pro,8.0,False,False
29,S-c75159,A-bcf87c,2024-08-22,2024-09-28,Enterprise,31,6169,74028,False,False,...,A-b20d99,Company_29,HealthTech,IN,2023-09-27,other,Basic,20.0,False,False
52,S-aff55c,A-bb2f49,2024-01-14,2024-12-29,Pro,8,0,0,True,False,...,A-02fac6,Company_52,FinTech,US,2023-04-06,event,Pro,22.0,False,False
73,S-0267b7,A-65a46c,2024-10-17,2024-12-04,Enterprise,13,0,0,True,False,...,A-9174e0,Company_73,Cybersecurity,US,2024-12-04,partner,Basic,37.0,False,False
78,S-ce5450,A-0a282f,2024-12-25,2024-12-28,Pro,8,0,0,True,False,...,A-432483,Company_78,Cybersecurity,US,2023-02-06,ads,Pro,4.0,False,False
86,S-09cdac,A-82861f,2024-07-31,2024-09-04,Basic,41,779,9348,False,False,...,A-40a557,Company_86,Cybersecurity,AU,2023-12-03,partner,Pro,16.0,True,False
